In [32]:
import os
import csv
from enum import Enum
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.remote.webelement import WebElement

In [33]:
class FileHandler():
    @staticmethod
    def is_file_empty(file_name: str) -> bool:
        """Return True if file is empty

        Args: 
            - file_name: file's name that is needed to be check
        """
        return os.stat(file_name).st_size == 0


    @staticmethod
    def write_to_csv(rows, file_name: str) -> None:
        header = ['Title', 'Content', 'Date', 'Url', 'Summary']
        file = open(file_name, 'a', encoding='UTF8', newline='')
        writer = csv.writer(file)

        if FileHandler.is_file_empty(file_name):
            writer.writerow(header)
        for row in rows:
            writer.writerow(row)

In [34]:
class BrowserOption(Enum):
    """Option for webbrowser
    """
    EDGE = 1
    CHROME = 2
    FIREFOX = 3
    SAFARI = 4

class TuoiTre_Crawler:
    @staticmethod
    def get_driver(browser_option: BrowserOption = BrowserOption.EDGE):
        """Return driver depended on BrowserOption Enum
        
        Args:
            - browser_option: the option of browser's driver
        """
        if browser_option == BrowserOption.EDGE:
            options = webdriver.EdgeOptions()
            options.add_argument("--blink-settings=imagesEnabled=false")
            options.add_experimental_option("prefs", {"profile.managed_default_content_settings.images": 2})
            return webdriver.ChromiumEdge(options=options)
        elif browser_option == BrowserOption.FIREFOX:
            return webdriver.Firefox()
        elif browser_option == BrowserOption.SAFARI:
            return webdriver.Safari()       
        else:
            options = webdriver.ChromeOptions()
            options.add_argument("--blink-settings=imagesEnabled=false")
            options.add_experimental_option("prefs", {"profile.managed_default_content_settings.images": 2})
            return webdriver.Chrome(options=options)  
        
    def __init__(self, browser_option: BrowserOption, file_name_to_save: str, category_url: list) -> None:
        """Create a new instance of crawler
        
        Args:
            - browser_option: the option of browser's driver
            - file_name_to_save: file's name to save data crawled from links in url_file
            - n_sample: maximum  can be crawled. If n_sample=None, no limited.
        """
        self.driver = TuoiTre_Crawler.get_driver(browser_option)
        # self.url_file = url_file
        self.file_name_to_save = file_name_to_save
        self.category_url = category_url

    def crawl(self, url: str):
        """Crawl data from single url
        
        Args:
            - url: a news url
        """
        self.driver.get(url)

        title_selector = "#main-detail > .article-title"
        contents_selector = "div.detail-content.afcbc-body > :not(.VCSortableInPreviewMode, #InreadPc)"
        date_selector = "#main-detail > div.detail-top > div.detail-time"
        summary_selector = "#main-detail > .detail-sapo"
        
        title = self.driver.find_element(By.CSS_SELECTOR, title_selector)
        contents = self.driver.find_elements(By.CSS_SELECTOR, contents_selector)
        date = self.driver.find_element(By.CSS_SELECTOR, date_selector)
        summary = self.driver.find_element(By.CSS_SELECTOR, summary_selector)

        title = title.text[:title.text.find('\n')]
        joined_content = " ".join(x.text for x in contents)
        return (title, joined_content, date.text, summary.text)

    def news_from_category(self, url: str):
        """Get news URL from category page
        
        Args:
            - url: a category page url
        """
        self.driver.get(url)

        focus_main_selector = "div.list__focus-main a.box-category-link-title"
        focus_main = self.driver.find_elements(By.CSS_SELECTOR, focus_main_selector)

        listing_main_selector = "div.list__listing-main a.box-category-link-title"
        listing_main = self.driver.find_elements(By.CSS_SELECTOR, listing_main_selector)

        news_urls = [url.get_property('href') for url in focus_main]
        news_urls.extend([url.get_property('href') for url in listing_main])
        
        rows = []
        for news_url in news_urls:
            try:
                (title, joined_content, date, summary) = self.crawl(news_url)
                rows.append([title, joined_content, date, news_url, summary])
            except:
                continue
        
        FileHandler.write_to_csv(rows, self.file_name_to_save)
        print("Crawled", len(rows), "from", url)
        return len(rows)

    def start_crawl(self):
        count = 0
        for url in self.category_url:
            count += self.news_from_category(url)

        print("Done! Crawled", count)
        self.driver.quit()

In [36]:
category_url = [
    "https://tuoitre.vn/kinh-doanh.htm",
    "https://tuoitre.vn/giai-tri.htm",
    "https://tuoitre.vn/the-thao.htm",
    "https://tuoitre.vn/giao-duc.htm"
]

crawler = TuoiTre_Crawler(BrowserOption.EDGE, file_name_to_save='tuoitre.csv', category_url=category_url)
crawler.start_crawl()

Crawled 29 from https://tuoitre.vn/kinh-doanh.htm
Crawled 28 from https://tuoitre.vn/giai-tri.htm
Crawled 28 from https://tuoitre.vn/the-thao.htm
Crawled 28 from https://tuoitre.vn/giao-duc.htm
Done! Crawled 113


### test

In [ ]:
def crawl(url: str) -> tuple[list[WebElement], list[WebElement]]:
    """Crawl data from single url
    
    Args:
        - url: a news url
    """
    options = webdriver.EdgeOptions()
    options.add_argument("--blink-settings=imagesEnabled=false")
    options.add_experimental_option("prefs", {"profile.managed_default_content_settings.images": 2})
    driver = webdriver.ChromiumEdge(options=options)

    driver.get(url)

    title_selector = "#main-detail > .article-title"
    title = driver.find_element(By.CSS_SELECTOR, title_selector)
    print(title.text)

    date_selector = "#main-detail > div.detail-top > div.detail-time"
    date = driver.find_element(By.CSS_SELECTOR, date_selector)
    print(date.text)

    summary_selector = "#main-detail > .detail-sapo"
    summary = driver.find_element(By.CSS_SELECTOR, summary_selector)
    print(summary.text)

    content_selector = "div.detail-content.afcbc-body > :not(.VCSortableInPreviewMode, #InreadPc)"
    content = driver.find_elements(By.CSS_SELECTOR, content_selector)
    joined_content = " ".join(x.text+'\n' for x in content)
    print("[", joined_content, "]")

crawl('https://tuoitre.vn/graffiti-bua-bai-nhech-nhac-o-bai-bien-da-nang-20240404152519117.htm')

Graffiti bừa bãi, nhếch nhác ở bãi biển Đà Nẵng
04/04/2024 15:38 GMT+7
Nhiều bờ tường, kè ở bãi biển Sơn Thủy thuộc quận Ngũ Hành Sơn, Đà Nẵng đang bị những hình vẽ graffiti làm nhếch nhác.
[ Thời gian gần đây, các bờ tường dọc khu nghỉ dưỡng lớn ở khu vực biển Sơn Thủy (Đà Nẵng) trở nên xấu xí bởi những hình vẽ nguệch ngoạc.
 Đáng chú ý, các bờ kè ven bãi biển này cũng bị bôi bẩn bởi những hình vẽ, chữ viết bằng sơn nhếch nhác.
 Nhiều người dân và du khách khi đi dọc lối xuống biển Sơn Thủy đã không khỏi lắc đầu ngao ngán.
 Anh Nguyễn Thành Vũ (43 tuổi) - một người dân ở phường Hòa Hải (quận Ngũ Hành Sơn, Đà Nẵng), lắc đầu ngao ngán khi nhìn những hình vẽ mất mỹ quan ở các bờ tường ven biển này.
 "Nếu là hình ảnh đẹp và phù hợp với cảnh biển thì ai cũng thích. Nhưng đây có bức thì chữ viết, bức thì mặt quỷ, có những bức bôi vài nét không rõ hình gì… Rất xấu xí", anh Vũ nói.
 Một người làm quản lý khu nghỉ dưỡng ở đây cho biết ban đầu bờ tường bên ngoài khu nghỉ dưỡng được vẽ khá chỉn 

In [20]:
def news_from_category(url: str):
    """Get news URL from category page
    
    Args:
        - url: a category page url
    """
    options = webdriver.EdgeOptions()
    options.add_argument("--blink-settings=imagesEnabled=false")
    options.add_experimental_option("prefs", {"profile.managed_default_content_settings.images": 2})
    driver = webdriver.ChromiumEdge(options=options)

    driver.get(url)

    focus_main_selector = "div.list__focus-main a.box-category-link-title"
    focus_main = driver.find_elements(By.CSS_SELECTOR, focus_main_selector)
    print("focus_main", len(focus_main))
    # for e in focus_main:
    #     print(e.get_property('href'))

    listing_main_selector = "div.list__listing-main a.box-category-link-title"
    listing_main = driver.find_elements(By.CSS_SELECTOR, listing_main_selector)
    print("listing_main", len(listing_main))
    # for e in listing_main:
    #     print(e.get_property('href'))
    
    news_url = [url.get_property('href') for url in focus_main]
    news_url.extend([url.get_property('href') for url in listing_main])
    print(len(news_url))
    for e in news_url:
        print(e)

news_from_category('https://tuoitre.vn/the-gioi.htm')

focus_main 9
listing_main 20
29
https://tuoitre.vn/can-cu-quan-su-o-thu-do-myanmar-bi-tap-kich-20240404144952755.htm
https://tuoitre.vn/nhung-cai-ten-la-ma-quen-moi-gia-nhap-lang-ti-phu-the-gioi-20240404125855054.htm
https://tuoitre.vn/ong-zelensky-noi-nga-chuan-bi-dua-them-300-000-quan-den-ukraine-20240404134946138.htm
https://tuoitre.vn/ong-trump-dan-truoc-ong-biden-o-6-bang-chien-dia-20240404113615727.htm
https://tuoitre.vn/ukraine-to-nga-tan-cong-don-dap-vung-kharkov-khien-4-nguoi-chet-20240404102740892.htm
https://tuoitre.vn/han-quoc-ong-lao-bi-thuong-da-chet-sau-khi-3-benh-vien-tu-choi-tiep-nhan-20240404092346951.htm
https://tuoitre.vn/nga-khuyen-phap-tu-bo-suy-nghi-gui-quan-den-ukraine-20240404101350237.htm
https://tuoitre.vn/dong-dat-o-dai-loan-so-nguoi-bi-thuong-vuot-moc-1-000-71-tho-mo-mac-ket-20240404092400243.htm
https://tuoitre.vn/my-lap-lung-chuyen-nha-nuoc-palestine-doc-lap-2024040408255724.htm
https://tuoitre.vn/tong-thong-macron-buc-xuc-to-nga-doa-dam-phap-trong-cuoc-d

In [2]:
import pandas as pd

csv = pd.read_csv('tuoitre.csv')
csv.head(10)

,Title,Content,Date,Url,Summary
0,Tăng trưởng tín dụng tại nhiều ngân hàng đã th...,"Tại đại hội cổ đông được tổ chức hôm nay 4-4, ...",04/04/2024 20:52 GMT+7,https://tuoitre.vn/tang-truong-tin-dung-tai-nh...,"Kết thúc quý 1, nhiều ngân hàng ghi nhận mức t..."
1,"Giá xăng RON95 giảm... 10 đồng, dầu có xu hướn...","Cụ thể, xăng E5RON92 có giá 23.910 đồng/lít, s...",04/04/2024 14:42 GMT+7,https://tuoitre.vn/gia-xang-ron95-giam-10-dong...,"Chiều 4-4, Bộ Công Thương công bố điều hành gi..."
2,Phát hiện 60.000 con tôm hùm giống nhập lậu qu...,"Ngày 4-4, Cục Hải quan thành phố Đà Nẵng cho b...",04/04/2024 14:49 GMT+7,https://tuoitre.vn/phat-hien-60-000-con-tom-hu...,Cục Hải quan thành phố Đà Nẵng cho biết vừa ph...
3,Thêm dự án 1.500 căn hộ ở TP.HCM được gỡ vướng...,UBND TP Thủ Đức vừa có quyết định số 4498 chấp...,04/04/2024 13:43 GMT+7,https://tuoitre.vn/them-du-an-1-500-can-ho-o-t...,UBND TP Thủ Đức vừa có quyết định chấp thuận đ...
4,"Người dân săn tour kích cầu, loạt deal được ch...","Ngay từ 9h ngày 4-4, các gian hàng trong khu v...",04/04/2024 16:15 GMT+7,https://tuoitre.vn/nguoi-dan-san-tour-kich-cau...,"Sáng 4-4, ngay khi Ngày hội Du lịch TP.HCM 202..."
5,25 trái chủ đồng ý đổi khoản nợ 284 triệu USD ...,"Ngày 4-4, Công ty CP Đầu tư Địa ốc No Va (Nova...",04/04/2024 10:21 GMT+7,https://tuoitre.vn/25-trai-chu-dong-y-doi-khoa...,"25 trái chủ, đại diện cho số dư nợ 284 triệu U..."
6,Dẹp loạn 'thổi giá' bất động sản,"Vì thế, cần phải sớm có sự vào cuộc của cơ qua...",04/04/2024 09:25 GMT+7,https://tuoitre.vn/dep-loan-thoi-gia-bat-dong-...,Khó khăn vẫn bủa vây ngành bất động sản nhưng ...
7,Điều gì đẩy giá USD ngân hàng liên tục phá đỉn...,Tỉ giá liên tục duy trì xu hướng tăng từ đầu n...,04/04/2024 09:00 GMT+7,https://tuoitre.vn/dieu-gi-day-gia-usd-ngan-ha...,"Ngoài áp lực thị trường thế giới, cầu tín dụng..."
8,"Khoai lang vào vụ, giá rơi xuống 3.500 đồng/kg","Tại Gia Lai, giá mua khoai lang vàng Nhật Bản ...",04/04/2024 08:54 GMT+7,https://tuoitre.vn/khoai-lang-vao-vu-gia-roi-x...,Nhiều vùng trồng khoai lang tại Tây Nguyên đan...
9,Đại sứ EU tại Việt Nam: Đã đến giai đoạn có th...,"Ngày 4-4, Bộ trưởng Bộ Nông nghiệp và Phát tri...",04/04/2024 21:43 GMT+7,https://tuoitre.vn/dai-su-eu-tai-viet-nam-da-d...,Theo đại sứ Liên minh châu Âu (EU) tại Việt Na...
